# CME193 - Homework 2

In this assignment, you'll be analyzing an interesting dataset: the passengers of the ship *Titanic*. As you are probably familiar, the *Titanic* was a ship that collided with an iceberg on April 15, 1912 and sank. About one-third of the passengers survived. We are interested in analyzing what factors are correlated with whether a person survived (whether the person was traveling with family, the person's sex, the person's age, etc.).

## Question 1: Loading the dataset    

The dataset contains the following columns:
- `Survived`: `1` if the person survived, `0` if not
- `Pclass`: the ticket class the person was travelling under, `1` for 1st class, `2` for 2nd class, `3` for 3rd class
- `Name`: name
- `Sex`: sex, `male` for male, `female` for female
- `Age`: age
- `Siblings/Spouses Aboard`: the number of siblings and spouses aboard
- `Parents/Children Aboard`: the number of parents and children aboard
- `Fare`: the amount paid for the ticket, in pounds

Eventually, we would like to use a classification algorithm to predict the `Survived` column from the other columns (besides `Name`). This means we need to convert the non-numeric columns into numeric columns (or boolean columns, for which `True` and `False` can be interpreted as `1` and `0` respectively).

You can find the dataset at the following URL (from CS 109, originally from Kaggle):

    http://web.stanford.edu/class/archive/cs/cs109/cs109.1166/stuff/titanic.csv

Load this CSV file into a Pandas dataframe and look at the data. Convert `Pclass` and `Sex` into boolean columns; that is, create four new boolean columns, `Female`, `1st Class`, `2nd Class`, and `3rd Class`, with the appropriate values. For example, `Female` would be `True` if the person is female, `False` if the person is male.



In [115]:
import pandas as pd
df = pd.read_csv('http://web.stanford.edu/class/archive/cs/cs109/cs109.1166/stuff/titanic.csv')

# TODO: YOUR CODE HERE
df['Female'] = df['Sex'] == 'female'
df['1st Class'] = df['Pclass'] == 1
df['2nd Class'] = df['Pclass'] == 2
df['3rd Class'] = df['Pclass'] == 3

df.head()

,Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare,Female,1st Class,2nd Class,3rd Class
0,0,3,Mr. Owen Harris Braund,male,22.0,1,0,7.2500,False,False,False,True
1,1,1,Mrs. John Bradley (Florence Briggs Thayer) Cum...,female,38.0,1,0,71.2833,True,True,False,False
2,1,3,Miss. Laina Heikkinen,female,26.0,0,0,7.9250,True,False,False,True
3,1,1,Mrs. Jacques Heath (Lily May Peel) Futrelle,female,35.0,1,0,53.1000,True,True,False,False
4,0,3,Mr. William Henry Allen,male,35.0,0,0,8.0500,False,False,False,True


## Question 2: Building the dataset
Next, let's convert our dataset into NumPy arrays. Create a NumPy array `X` derived from the Pandas dataframe with the numerical and boolean columns, that is: `['Age', 'Siblings/Spouses Aboard', 'Parents/Children Aboard', 'Fare', 'Female', '1st Class', '2nd Class', '3rd Class']`. Create a NumPy vector `y` derived from the `Survived` column. Ensure that the entries of both `X` and `y` are floating point numbers. (Hint: if `a` is a NumPy array, then `a.astype(float)` converts the entries to floats.)

In [116]:
# TODO: YOUR CODE HERE
X = df[['Age', 'Siblings/Spouses Aboard', 'Parents/Children Aboard', 'Fare', 'Female', '1st Class', '2nd Class', '3rd Class']].astype(float)
y = df['Survived'].astype(float)
X.head()

,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare,Female,1st Class,2nd Class,3rd Class
0,22.0,1.0,0.0,7.2500,0.0,0.0,0.0,1.0
1,38.0,1.0,0.0,71.2833,1.0,1.0,0.0,0.0
2,26.0,0.0,0.0,7.9250,1.0,0.0,0.0,1.0
3,35.0,1.0,0.0,53.1000,1.0,1.0,0.0,0.0
4,35.0,0.0,0.0,8.0500,0.0,0.0,0.0,1.0


## Question 3: Logistic regression
Logistic regression is a classification algorithm that comes with scikit-learn. In this case, we would like to predict one of two classes, `1` if the person survived, `0` if the person did not. Normally, we would split our dataset into a training set and a test set, but for simplicity we will not do that here; instead we will train on our entire dataset.

Using scikit-learn, fit a logistic regression model to the dataset we created in Question 2. What percentage of the passengers' outcomes does the model correctly predict? What does the model think about the fate of a 30-year-old male travelling alone who paid 50 pounds for his 2nd-class ticket?

In [117]:
# YOUR CODE HERE
from sklearn import linear_model
import numpy as np

model = linear_model.LogisticRegression()

model.fit(X, y)

perc_corr_pred = model.score(X, y) * 100

man_model = np.array([30, 0, 0, 50, False, False, True, False]).reshape(1,-1)
man_model.astype(float)

if model.predict(man_model) == 0:
    txt = "not survived"
elif model.predict(man_model) == 1:
    txt = "survived"

print("The model predicts {} % of the passengers' outcomes correctly".format(round(perc_corr_pred, 2)))
print("The 30-year old man has " + txt + " according to the model")

The model predicts 80.38 % of the passengers' outcomes correctly
The 30-year old man has not survived according to the model


/Users/albanbroze/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


## Question 4: Defining the model
As an exercise, we will now do logistic regression again, but this time "manually" -- without the use of scikit-learn -- by solving the underlying optimization problem. To do so, we'll make use of SciPy's `special` and `optimize` packages.

Consider a single observation $x$ (a single passenger), represented by a vector of length $k$. Logistic regression defines a model with two parameters, $\alpha$ and $\beta$, where $\alpha$ is a number and $\beta$ is a vector of length $k$. Assuming we know $\alpha$ and $\beta$, the probability that $x$ survives is the number
$$ \text{probability that }x\text{ survives} = \frac{1}{1 + \exp(-(\alpha + x^T \beta)) }.$$
Our eventual goal is to find the values for $\alpha$ and $\beta$ that results in a probability that best matches the observed outcome for $x$.

Define a function `probability_of_surviving(alpha, beta, X)` that computes probabilities of the passengers in `X` surviving,
$$ \frac{1}{1 + \exp(-(\alpha + X \beta )) },$$
where:
- `alpha` is a number $\alpha$
- `beta` is a vector $\beta$ of length $k$
- `X` is an $n$-by-$k$ matrix $X$, where each of the $n$ rows corresponds to an observation (a passenger), and each column corresponds to a feature.

This function should output a vector of length $n$, with each entry being the probability that each person survives, assuming we know $\alpha$ and $\beta$. Note that $X\beta$ should be interpreted as matrix multiplication, but all other operations (addition, exponential, division) operate elementwise. (Hint: check out SciPy's `special.expit`.)

In [118]:
# TODO: implemenet the function below

from scipy import special, optimize

def probability_of_surviving(alpha, beta, X):
    return special.expit(alpha + X@beta)

## Question 5: Defining the loss
Our goal is to find the $\alpha$ and $\beta$ values that best match the observed data $(X, y)$. To do this, we will construct a loss function that, when given $\alpha$ and $\beta$, characterizes how good our predictions $\hat{y}$ are compared to the ground truth $y$. 

Mathematically, our loss function will be $$
    L(\alpha, \beta, X, y) = \text{sum}(\text{KL}(y, \hat{y})) + \frac{1}{2}\beta^T \beta,
$$ where:
- $\hat{y}$ is the vector of predicted probabilities (from Question 4), which is computed from $\alpha$, $\beta$, and $X$
- $\text{KL}(y, \hat{y})$ is the *Kullback-Liebler divergence*, which measures how different the ground truth $y$ is from the predicted probabilities $\hat{y}$
- $\text{sum}(\cdot)$ sums up the entries of a vector, in this case adding up the loss for each passenger in $X$
- $\frac{1}{2}\beta^T \beta$ is a *regularization term* that prevents overfitting.

Define a function `logistic_regression_loss(alpha_beta, X, y)` that computes $L(\alpha, \beta, X, y)$, where
- `alpha_beta` is a vector of length $1+ k$ that contains $\alpha$ in its first entry and $\beta$ in the remaining entries
- `X` is an $n$-by-$k$ matrix $X$, where each of the $n$ rows corresponds to an observation (a passenger), and each column corresponds to a feature
- `y` is a vector $y$ of length $n$, that is 1 if the passenger survived and 0 if they did not.

(Hint: check out SciPy's `special.kl_div`.) 

In [119]:
# TODO: Implement functions below
def logistic_regression_loss(alpha_beta, X, y):
    
    alpha = alpha_beta[0]
    beta = alpha_beta[1:]
    
    y_pred = probability_of_surviving(alpha, beta, X)
    
    loss = np.sum(special.kl_div(y, y_pred)) + 1/2 * np.linalg.norm(beta)**2
    
    return loss

## Question 6: Logistic regression (the hard way)
Use SciPy's `optimize.minimize` to find the $\alpha$ and $\beta$ that best explain the data $(X,y)$. In other words, find $\alpha$ and $\beta$ that minimizes the function `logistic_regression_loss`. Use an initial guess of $\alpha = 0$, $\beta = 0$. Compare your result to the $\alpha$ and $\beta$ computed by scikit-learn, given by `model.intercept_` and `model.coef_`. (It will not match exactly but should be somewhat close.)

(Hint: you'll need to make use of the `args` argument for `optimize.minimize` to pass in `X` and `y`. Also, you may get a warning that the desired error was not achieved, but you can ignore this.)



In [120]:
# TODO: use optimize.minimize to implement the logistic regression
# AND compare the result to Q3 using scikit-learn

x0 = np.zeros(len(X.columns)+1)
res = optimize.minimize(logistic_regression_loss, x0=x0, args=(X,y))

print("model 1 alpha: ", model.intercept_[0])
print("model 2 alpha: ", res.x[0])
print()
print("model 1 beta: ", model.coef_[0])
print("model 2 beta: ", res.x[1:])

model 1 alpha:  0.10352360064850934
model 2 alpha:  0.12634090109401394

model 1 beta:  [-0.04161514 -0.38656389 -0.09896949  0.00327638  2.63675698  1.12922887
  0.06289571 -1.08233042]
model 2 beta:  [-0.04302206 -0.36806956 -0.06156255  0.00386969  2.32004739  1.03282188
 -0.1387026  -0.89412047]


## Question 7: Predictions

With what probability does the model learned in Question 6 think a 30-year-old male travelling alone who paid 50 pounds for his 2nd-class ticket will survive? What percentage of the passengers' outcomes does the model correctly predict, if we say that the model predicts survival if the probability of survival is greater than 0.5?

In [121]:
# TODO: Implement your code here
alpha = res.x[0]
beta = res.x[1:]
X_man = np.array([30, 0, 0, 50, False, False, True, False]).astype(float)

print("The model thinks the man will survive with probability {} %".format(round(probability_of_surviving(alpha, beta, X_man)*100,2)))

pred = probability_of_surviving(alpha, beta, X)
pred[pred > 0.5] = True
pred[pred <= 0.5] = False
pred_eval = (y == pred.astype(float))
pred_eval_perc = np.count_nonzero(pred_eval)/len(pred_eval) * 100

print("The model predicts {} % of the passengers' outcomes correctly".format(round(pred_eval_perc, 2)))

The model thinks the man will survive with probability 24.8 %
The model predicts 80.61 % of the passengers' outcomes correctly


# Submission instructions

Get a sharable link that can be viewed by anyone with the link, and submit it on gradescope.